# Skeletonization using meshparty
* Download meshparty from here: https://meshparty.readthedocs.io/en/latest/

In [16]:
from meshparty import trimesh_io,trimesh_vtk,utils, skeletonize
from annotationframeworkclient import FrameworkClient
from pathlib import Path
import json
import numpy as np
import pandas as pd
from cloudvolume import CloudVolume
from concurrent import futures


## Setup credentials, clients, and mesh meta object

In [4]:
# Get tokens and cloudpaths
with open(Path.home() / 'cloudvolume' / 'secrets'/'chunkedgraph-secret.json') as f:
        tokens = json.load(f)

with open(Path.home() / 'cloudvolume' / 'segmentations.json') as f:
            cloud_paths = json.load(f)

        
cv = CloudVolume(cloud_paths['Dynamic_V1']['url'],use_https=True,secrets=tokens['dev'])

# Get client object      
dev_token = tokens['dev']
auth_token = tokens['api']
datastack_name = 'vnc_v0' # from https://api.zetta.ai/wclee/info/
client = FrameworkClient(
    datastack_name,
    server_address = "https://api.zetta.ai/wclee",
    auth_token = auth_token
)


## Methods for downloading annotation tables

In [5]:
def download_annotation_table(client,table_name,ids=range(1000)):
    entries = client.annotation.get_annotation(table_name,ids)
    annotation_table = pd.DataFrame(entries)
    return(annotation_table)


def seg_from_pt(pts,vol,image_res=np.array([4.3,4.3,45]),max_workers=4):
    ''' Get segment ID at a point. Default volume is the static segmentation layer for now. 
    Args:
        pts (list): list of 3-element np.arrays of MIP0 coordinates
        vol_url (str): cloud volume url
    Returns:
        list, segment_ID at specified point '''
    
    
    seg_mip = vol.scale['resolution']
    res = seg_mip / image_res

    pts_scaled = [pt // res for pt in pts]
    results = []
    with futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
        point_futures = [ex.submit(lambda pt,vol: vol[list(pt)][0][0][0][0], k,vol) for k in pts_scaled]
        
        for f in futures.as_completed(point_futures):
            results=[f.result() for f in point_futures]
       
    return results



def generate_soma_table(annotation_table,
                        segmentation_version='Dynamic_V1',
                        resolution=np.array([4.3,4.3,45]),
                        token=None):
    ''' Generate a soma table used for microns analysis. This is the workaround for a materialization engine
    Args:
        annotation_table: pd.DataFrame, output from download_cell_table. Retreived from the annotation engine.
        segmentation_version: str, Currently we have 4 for FANC. Two flat segmentations ("Flat_1" and "Flat_2") and two dynamic ("Dynamic_V1/V2"). 
                              This will only work if you have a segmentations.json in your cloudvolume folder. See examples for format.
        resolution: np.array, Resolution of the mip0 coordinates of the version (not necessarily the same as the segmentation layer resolution).
                              For all but the original FANC segmentation, this will be [4.3,4.3,45]
        token: str, currently, CloudVolume requires a workaround for passing google secret tokens. This won't work unless you edit your cloudvolume 
                              file to remove the check for hexidecimal formatting of tokens. Updates should be coming to fix this. 
        '''

    soma_table = pd.DataFrame(columns=['name','cell_type',
                                       'pt_position','pt_root_id',
                                       'soma_x_nm','soma_y_nm','soma_z_nm',
                                       'found'])
    with open(Path.home() / 'cloudvolume' / 'segmentations.json') as f:
            cloud_paths = json.load(f)
    if 'Dynamic' in segmentation_version:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'],agglomerate=True,use_https=True,secrets=token)
    else:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'])
        
    seg_ids = seg_from_pt(annotation_table.pt_position,cv)
    
    soma_table.name = annotation_table.tag
    soma_table.pt_position = annotation_table.pt_position
    soma_table.pt_root_id = seg_ids
    soma_table.soma_x_nm = np.array([i[0] for i in annotation_table.pt_position]) * resolution[0]
    soma_table.soma_y_nm = np.array([i[1] for i in annotation_table.pt_position]) * resolution[1]
    soma_table.soma_z_nm = np.array([i[2] for i in annotation_table.pt_position]) * resolution[2]
    
    return(soma_table)



def generate_synapse_table(annotation_table,
                        segmentation_version='Dynamic_V1',
                        resolution=np.array([4.3,4.3,45]),
                        token=None):
    ''' Generate a soma table used for microns analysis. This is the workaround for a materialization engine
    Args:
        annotation_table: pd.DataFrame, output from download_cell_table. Retreived from the annotation engine.
        segmentation_version: str, Currently we have 4 for FANC. Two flat segmentations ("Flat_1" and "Flat_2") and two dynamic ("Dynamic_V1/V2"). 
                              This will only work if you have a segmentations.json in your cloudvolume folder. See examples for format.
        resolution: np.array, Resolution of the mip0 coordinates of the version (not necessarily the same as the segmentation layer resolution).
                              For all but the original FANC segmentation, this will be [4.3,4.3,45]
        token: str, currently, CloudVolume requires a workaround for passing google secret tokens. This won't work unless you edit your cloudvolume 
                              file to remove the check for hexidecimal formatting of tokens. Updates should be coming to fix this. 
        '''
     
    
    synapse_table = pd.DataFrame(columns=['id','pre_root_id','post_root_id',
                                      'cleft_vx','ctr_pt_x_nm','ctr_pt_y_nm','ctr_pt_z_nm',
                                      'pre_pos_x_vx','pre_pos_y_vx','pre_pos_z_vx',
                                      'ctr_pos_x_vx','ctr_pos_y_vx','ctr_pos_z_vx',
                                      'post_pos_x_vx','post_pos_y_vx','post_pos_z_vx'])

    with open(Path.home() / 'cloudvolume' / 'segmentations.json') as f:
            cloud_paths = json.load(f)
    if 'Dynamic' in segmentation_version:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'],agglomerate=True,use_https=True,secrets=token)
    else:
        cv = CloudVolume(cloud_paths[segmentation_version]['url'])
        
    pre_ids = seg_from_pt(annotation_table.pre_pt_position,cv)
    post_ids = seg_from_pt(annotation_table.post_pt_position,cv)
    
    synapse_table.pre_root_id = pre_ids
    synapse_table.post_root_id = post_ids
    
    # TODO: This in not a stupid way. 
    synapse_table.ctr_pt_x_nm = np.array([i[0] for i in annotation_table.ctr_pt_position]) * resolution[0]
    synapse_table.ctr_pt_y_nm = np.array([i[1] for i in annotation_table.ctr_pt_position]) * resolution[1]
    synapse_table.ctr_pt_z_nm = np.array([i[2] for i in annotation_table.ctr_pt_position]) * resolution[2]
    
    synapse_table.pre_pos_x_vx = np.array([i[0] for i in annotation_table.pre_pt_position]) 
    synapse_table.pre_pos_y_vx = np.array([i[1] for i in annotation_table.pre_pt_position]) 
    synapse_table.pre_pos_z_vx = np.array([i[2] for i in annotation_table.pre_pt_position]) 
    
    synapse_table.post_pos_x_vx = np.array([i[0] for i in annotation_table.post_pt_position]) 
    synapse_table.post_pos_y_vx = np.array([i[1] for i in annotation_table.post_pt_position]) 
    synapse_table.post_pos_z_vx = np.array([i[2] for i in annotation_table.post_pt_position]) 
    
    return(synapse_table)
    
    

In [10]:
print(client.annotation.get_tables())
syn_table = '81A07_Synapses'
cel_table = 'T1MN_somas'



['test_synapse_table', 'test_synapse_table_2', 'test_T1MN_soma_table', 'T1MN_somas', '10B_somas', 'claw_neurons', '81A07_Synapses', 'test_table', 'club_synapses', 'claw_neuron_synapses', 'test']


In [11]:
cells_raw = download_annotation_table(client,cel_table)
syns_raw = download_annotation_table(client,syn_table)

synapse_table = generate_synapse_table(syns_raw,token=dev_token)
soma_table = generate_soma_table(cells_raw,token=dev_token)


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.81it/s]

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.08it/s]A

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.91it/s]


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]



Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.90it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.85it/s]A



Downloading:   0%|          | 0/1 [00:0

In [15]:
synapse_table.loc[synapse_table.post_root_id==648518346677949170]

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
27,NaN,648518346702967118,648518346677949170,NaN,205742.1,470794.1,106110.0,47831,109470,2358,NaN,NaN,NaN,47864,109505,2358


synapse_tables for skeletonization
 - Meshparty can be used to associate our synapses with a skeleton, even if it is not ideal for manual inspection. 
 - The other issue with meshparty is that you need to supply soma coordinates, you cannot just skeletonize using a seg_id from a synapse point. This means that you need both a cell table and a synapse table if you want to skeletonize partners. 

In [ ]:
def get_mesh(cv,seg_id):
    
    if 'graphene' in cv.meta.info['mesh']:
        mesh = cv.mesh.get(seg_id,use_byte_offsets=True)[seg_id]
    else:
        mesh = cv.mesh.get(seg_id,use_byte_offsets=False)
    
    mp_mesh =trimesh_io.Mesh(mesh.vertices,mesh.faces)

    return(mp_mesh)


def meshparty_skeletonize(mesh,
                   soma_coords,
                   NG_voxel_resolution = np.array([4.3,4.3,45]),
                   merge_size_threshold=100,
                   merge_max_dist=1000,
                   merge_distance_step=200,
                   soma_radius=12000,
                   invalidation_distance=5500,
                   merge_collapse_soma=True):
       
    adjusted_soma = soma_coords * NG_voxel_resolution
    # Repair mesh
    try:
        mesh.merge_large_components(size_threshold=merge_size_threshold,
                                    max_dist=merge_max_dist,
                                    dist_step=merge_distance_step)
    except:
        print('mesh heal failed')
    
    # Skeletonize
    skeleton =skeletonize.skeletonize_mesh(mesh,
                                     adjusted_soma,
                                     soma_radius=soma_radius,
                                     invalidation_d=invalidation_distance,
                                     collapse_soma=merge_collapse_soma)
    
                  
    return(skeleton)


In [ ]:
to_skeletonize = 'MN_A101_T1L'
mesh = get_mesh(cv,soma_table.loc[soma_table.name==to_skeletonize,'pt_root_id'].values[0],)   


In [ ]:
skeleton = meshparty_skeletonize(mesh = mesh,
                                 soma_coords=soma_table.loc[soma_table.name==to_skeletonize,['soma_x_nm','soma_y_nm','soma_z_nm']].values[0])

In [ ]:
mesh_actor = trimesh_vtk.mesh_actor(mesh,color = (1,0,0),opacity=.5)
skel_actor = trimesh_vtk.skeleton_actor(skeleton,color = (0,0,1))

In [ ]:
trimesh_vtk.render_actors([mesh_actor,skel_actor])